In [1]:
from tqdm import tqdm, trange
from libsvm.svmutil import svm_read_problem # https://blog.csdn.net/u013630349/article/details/47323883
from time import time

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


%matplotlib inline

## Plot functions

In [3]:
def plot_single(seq, xlabel='Iteration', ylabel='Gradient Norm', title=''):
    plt.figure()
    iterations = np.arange(len(seq))+1
    plt.semilogy(iterations,seq)
    plt.xticks(iterations)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

# 
def plot_multi_seqs(seqs, xlabel='Iteration', ylabel='Gradient Norm', title='', xtick_step = 50):
    plt.figure(figsize=(16,8), dpi=150)
    maxLen = 0
    for seq in seqs:
        iterations = seq.index
        if iterations.size > maxLen:
            maxLen = iterations.size
        plt.semilogy(iterations, seq, label=seq.name)
    plt.xticks(np.arange(stop=maxLen,step=xtick_step)+1)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

## Data reader

In [2]:
def read_data(path):
    b, A = svm_read_problem(path)
    rows = len(b)   # 矩阵行数, i.e. sample 数
    cols = max([max(row.keys()) if len(row)>0 else 0 for row in A])  # 矩阵列数, i.e. feature 数
    b = np.array(b)
    A_np = np.zeros((rows,cols))
    for r in range(rows):
        for c in A[r].keys():
            # MatLab 是 1-index, python 则是 0-index
            A_np[r,c-1] = A[r][c]
    # 清楚全 0 features
    effective_row_ids = []
    for idx, row in enumerate(A_np):
        if np.sum(row) > 1e-3:
            effective_row_ids.append(idx)
    return b[effective_row_ids], A_np[effective_row_ids]

b, A = read_data('w8a')
# b, A = read_data('ijcnn1.test')
# b, A = read_data('a9a.test')
# b, A = read_data('CINA.test')
m,n = A.shape
m,n

(45546, 300)

## Related Functions

令 $g_i(x)=\exp(b_i x^\top a_i), i=1,2,...,m$

所以可以得到目标函数为:
$$
f(x)=\frac{1}{m}\sum_{i=1}^{m}\log\left( 1+ \frac{1}{g_i(x)}\right) + \frac{1}{100m}\left\|x\right\|^2
$$
进而可以得到梯度 $\nabla f(x)$ 和 Hessian矩阵 $\nabla^2 f(x)$ 的形式:
$$
\nabla f(x) = \frac{1}{m}\sum_{i=1}^{m}\left[ -b_i a_i (1+g_i(x))^{-1} \right] + \frac{1}{50m}x
$$
$$
\nabla^2 f(x) =  \frac{1}{m}\sum_{i=1}^{m}\left( b_i^2 \frac{g_i(x)}{(1+g_i(x))^2}a_i a_i^\top \right) + \frac{1}{50m}I
$$

此处函数内的 $a_i,b_i$ 和数据本身相关

In [3]:
x = np.array([10,100,1000,-1,-10])
exp_x = np.exp(x)
log1p_exp = np.log(1+exp_x)
log1p_exp
idxs = np.where(exp_x==float('inf'))
log1p_exp[idxs] = -x[idxs]
log1p_exp

/tmp/ipykernel_1673850/2359670605.py:2: RuntimeWarning: overflow encountered in exp
  exp_x = np.exp(x)


array([ 1.00000454e+01,  1.00000000e+02, -1.00000000e+03,  3.13261688e-01,
        4.53988992e-05])

In [4]:
def f(x):
    bAx = b*(A@x)
    exp_mbAx = np.exp(-bAx)
    log1p_exp = np.log(1+exp_mbAx)
    overflow_idxs = np.where(exp_mbAx==float('inf'))
    log1p_exp[overflow_idxs] = -bAx[overflow_idxs]
    # return np.log(1+np.exp(-bAx)).mean() + 1/(100*m)* x.T@x
    return log1p_exp.mean() + 1/(100*m)* x.T@x

def f_grad(x):
    # return np.ones(m)@(np.expand_dims((-b)/(1+np.exp(b*(A@x))), axis=1)*A)/m
    return np.ones(m)@(np.expand_dims((-b)/(1+np.exp(b*(A@x))), axis=1)*A)/m + 1/(50*m)*x

def f_hessian(x):
    Ax = A@x
    exp_bAx = np.exp(b*Ax)
    # return (A.T @ (np.expand_dims(b*b*exp_bAx/(1+exp_bAx)**2, axis=1)*A) )/m
    return (A.T @ (np.expand_dims(b*b*exp_bAx/(1+exp_bAx)**2, axis=1)*A) )/m + 1/(50*m)*np.diag([1.0]*x.size)

# f(np.ones(n),A,b)
# f(np.ones(n))
hessian = f_hessian(np.ones(n)*0)
# np.linalg.inv(hessian)
np.linalg.matrix_rank(hessian)

300

In [74]:
f0 = lambda x: np.log(1+np.exp(-b*(A@x))).mean() + 1/(100*m)* x.T@x
f0(np.zeros(n)+0.5)

6.283620768609435

In [75]:
f(np.zeros(n)+0.5)

6.283620768609435

## Baselines

In [11]:
np.random.seed(1234)
x = np.random.randn(10)
y = np.random.randn(10)
x.T@y
# x@y


5.255647111054592

### Gradient Descent

In [5]:
#* Armijo rule 
def armijo_search(f, f_grad, xk, t_hat, alpha, beta, D, isNewton=False, dk=None):
    if isNewton:
        assert dk is not None
    tk = t_hat*1
    grad = f_grad(xk)
    while True:
        if isNewton:
            if np.linalg.norm(xk+tk*dk,ord=2)<=D/2 and f(xk+tk*dk) <= f(xk) + alpha*tk*grad.T@dk:
                break
        else:
            if np.linalg.norm(xk-tk*grad,ord=2)<=D/2 and f(xk-tk*grad) <= f(xk)-alpha*tk*grad.T@grad:
                break
        tk *= beta
    return tk

#* 梯度下降法
def gradient_descent(f, f_grad, x0, D, t_hat=1, epsilon=1e-6, max_iters=10000):
    func_val_record = []
    grad_norm_record = []
    xk = x0
    t_s = time()
    # for idx in trange(max_iters):
    for idx in range(max_iters):
        tk = armijo_search(f, f_grad, xk, t_hat=t_hat, alpha=0.1, beta=0.5, D=D)
        xk_next = xk-tk*f_grad(xk)
        func_val_record.append(f(xk_next))
        grad_next = np.linalg.norm(f_grad(xk_next),ord=2)
        grad_norm_record.append(grad_next)
        # termination criteria
        if grad_next<=epsilon:
            break
        else:
            print(grad_next, tk, np.linalg.norm(xk_next))
        xk = xk_next
    t_e = time()
    return xk_next, np.asarray(func_val_record), np.asarray(grad_norm_record), t_e-t_s


In [33]:
np.random.seed(1000)

init_x = np.zeros(n)

x_opt, _, _, t = gradient_descent(f=f, f_grad=f_grad, x0=init_x, D=500, t_hat=2, epsilon=1e-4, max_iters=1000)
optim_val = f(x_opt)
print(f'最小值: {optim_val:>2f}\t耗时: {t:>2f}s')


0.14620341015625468 2 1.2288947117785496
0.11141037897550853 2 1.501753341092599
0.0911462896089828 2 1.7114450133017853
0.07768442262375062 2 1.88397571307895
0.06803171169890225 2 2.0314761194284343
0.06074591369917542 2 2.160802807103633
0.05503896266785273 2 2.2762644223683797
0.050441447671832916 2 2.3807650702417256
0.04665527232131667 2 2.4763640882859663
0.04348148098553537 2 2.564579505840086
0.04078189381381589 2 2.6465652036671172
0.03845736040218016 2 2.723220413058256
0.036434774360414805 2 2.795260554814229
0.03465897705294006 2 2.86326481719174
0.033087522317326624 2 2.927709129813346
0.0316871839310042 2 2.988989634950575
0.03043156191108267 2 3.04743978397356
0.02929940271431947 2 3.103343042894024
0.028273395596853615 2 3.1569425029513174
0.027339294031007697 2 3.2084482648819495
0.02648526366945201 2 3.258043192508028
0.025701391160557474 2 3.305887452444363
0.024979309099776994 2 3.3521221369335383
0.024311906116497105 2 3.3968721849552246
0.023693100241406313 2 3.4

In [34]:
f(x_opt)

0.08839043698796589

### Newton (Interior Point Methods)

logarithmic barrier:
$$
\phi(x)=-\log(-g(x)),\quad g(x)=\left\|x\right\|_2 - D/2
$$
$$
\nabla g(x) = x/\left\|x\right\|_2, \quad \nabla^2 g(x)= \left\|x\right\|_2^{-1}I-\left\|x\right\|_2^{-3}xx^\top
$$
gradient and hessian:
$$
\nabla \phi(x) = \frac{1}{-g(x)}\nabla g(x)=\frac{x/\left\|x\right\|_2}{D/2-\left\|x\right\|_2}=\frac{x}{\left\|x\right\|_2(D/2-\left\|x\right\|_2)}
$$
$$
\nabla^2\phi(x)=\frac{1}{g(x)^2}\nabla g(x)\nabla g(x)^\top + \frac{1}{-g(x)}\nabla^2 g(x) = \frac{1}{\left\|x\right\|_2(D/2-\left\|x\right\|_2)}I+\frac{2\left\|x\right\|_2-D/2}{\left\|x\right\|_2^3(D/2-\left\|x\right\|_2)^2}xx^\top
$$
central path - for $t>0$:
$$
\min_x tf(x)+\phi(x)
$$

In [13]:
def phi(x,D):
    return -np.log(D/2-np.linalg.norm(x,ord=2))

def phi_grad(x,D):
    x_norm = np.linalg.norm(x,ord=2)
    return x/(x_norm*(D/2-x_norm))

def phi_hessian(x,D):
    x_norm = np.linalg.norm(x,ord=2)
    xxT = np.matmul(x[:,None],x[None,:])    # x * xT
    return np.eye(x.size)/(x_norm*(D/2-x_norm)) + (2*x_norm-D/2)/(x_norm**3 * (D/2-x_norm)**2)*xxT


#* 外部迭代
def barrier_method(t_init, f, f_grad, f_hessian, phi, phi_grad, phi_hessian, A, b, x0, D, num_constraints, mu, epsilon=1e-6, maxIter=20):
    xt = x0
    t = t_init
    duality_gaps = []
    t_s = time()
    for i in range(maxIter):
        xt,num_newton_step = solve_central(f=lambda x:t*f(x)+phi(x,D), 
                                f_grad=lambda x:t*f_grad(x)+phi_grad(x,D), 
                                f_hessian=lambda x:t*f_hessian(x)+phi_hessian(x,D),
                                A=A, b=b, x0=xt, D=D, epsilon=epsilon)
        duality_gaps.extend([num_constraints/t]*num_newton_step)
        # print(f'**{i}**: f_val=',f(xt))
        if num_constraints/t < epsilon:
            break
        t *= mu
    t_e = time()
    return xt, t_e-t_s, duality_gaps

#* 阻尼牛顿
def solve_central(f, f_grad, f_hessian, A, b, x0, D, epsilon=1e-6, max_iter=50):
    xk = x0
    iter_cnt = 0
    for idx in range(max_iter):
        iter_cnt += 1
        grad = f_grad(xk)
        hessian = f_hessian(xk)
        dk = -np.linalg.inv(hessian)@grad
        decrement = (-grad@dk)**0.5
        if decrement**2/2 <= epsilon:
            return xk, iter_cnt
        tk = armijo_search(f, f_grad, xk, t_hat=1, alpha=0.1, beta=0.5, D=D, isNewton=True, dk=dk)
        print('Iter Cnt.:',iter_cnt, 'Decrement:',decrement, 'fval:',f(xk), 'tk:',tk)
        xk += tk*dk
    return xk, iter_cnt
        

In [15]:
np.random.seed(1000)
t_init = 1
x0 = np.zeros(n)+0.005
x_opt, t, duality_gaps = barrier_method(t_init=t_init, f=f, f_grad=f_grad, f_hessian=f_hessian, phi=phi, phi_grad=phi_grad, phi_hessian=phi_hessian, 
                A=A, b=b, x0=x0, D=500, num_constraints=1, mu=10, epsilon=1e-6, maxIter=20)
optim_val = f(x_opt)
print(f'最小值: {optim_val:>2f}\t耗时: {t:>2f}s')

Iter Cnt.: 1 Decrement: 0.8181937689877108 fval: -4.796761076282368 tk: 1
Iter Cnt.: 2 Decrement: 0.4311602474045324 fval: -5.18588281281812 tk: 1
Iter Cnt.: 3 Decrement: 0.2882439952273807 fval: -5.304571735466875 tk: 1
Iter Cnt.: 4 Decrement: 0.18771922818407033 fval: -5.357852773746775 tk: 1
Iter Cnt.: 5 Decrement: 0.10403976891343639 fval: -5.379962815361623 tk: 1
Iter Cnt.: 6 Decrement: 0.03879841180285344 fval: -5.3864071146154515 tk: 1
Iter Cnt.: 7 Decrement: 0.006160860143799874 fval: -5.387229984601458 tk: 1
Iter Cnt.: 1 Decrement: 0.5183003387252821 fval: -4.536631961588298 tk: 1
Iter Cnt.: 2 Decrement: 0.2680219618783167 fval: -4.700637786771958 tk: 1
Iter Cnt.: 3 Decrement: 0.1072814038208272 fval: -4.743498043202538 tk: 1
Iter Cnt.: 4 Decrement: 0.028160042842082057 fval: -4.749983844011594 tk: 1
Iter Cnt.: 5 Decrement: 0.003641964683180588 fval: -4.750409219543984 tk: 1
Iter Cnt.: 1 Decrement: 0.7812286409870396 fval: 0.9875108406537896 tk: 1
Iter Cnt.: 2 Decrement: 0.427

In [12]:
#* pure阻尼牛顿
def damped_newton(f, f_grad, f_hessian, x0, D, epsilon=1e-6, max_iters=100):
    func_val_record = []
    grad_norm_record = []
    xk = x0
    t_s = time()
    # for idx in trange(max_iters):
    for idx in range(max_iters):
        grad = f_grad(xk)
        # print(np.linalg.norm(grad))
        hessian = f_hessian(xk)
        # dk = -np.linalg.pinv(hessian)@grad
        dk = -np.linalg.inv(hessian)@grad
        tk = armijo_search(f, f_grad, xk, t_hat=1, alpha=0.1, beta=0.5, D=D, isNewton=True, dk=dk)
        xk_next = xk + tk*dk
        func_val_record.append(f(xk_next))
        # grad_norm_record.append(np.linalg.norm(f_grad(xk_next),ord=2))
        grad_next = np.linalg.norm(f_grad(xk_next),ord=2)
        grad_norm_record.append(grad_next)
        # termination criteria
        if grad_next<=epsilon:
            break
        else:
            print('grad_norm:',grad_next,"tk:",tk, "x_norm:",np.linalg.norm(xk_next))
        xk = xk_next
    t_e = time()
    return xk_next, np.asarray(func_val_record), np.asarray(grad_norm_record), t_e-t_s

np.random.seed(1000)

# init_x = np.zeros(n)+0.5
init_x = np.zeros(n)

x_opt, _, _, t = damped_newton(f=f, f_grad=f_grad, f_hessian=f_hessian, x0=init_x, D=500, epsilon=1e-8, max_iters=50)
optim_val = f(x_opt)
print(f'最小值: {optim_val:>2f}\t耗时: {t:>2f}s')

grad_norm: 0.1815888135199014 tk: 1 x_norm: 11.146322350101936
grad_norm: 0.07488480693510081 tk: 1 x_norm: 18.75318326380974
grad_norm: 0.03147405256953375 tk: 1 x_norm: 27.394693984410537
grad_norm: 0.012915384418461231 tk: 1 x_norm: 36.72297176961722
grad_norm: 0.004946900163369329 tk: 1 x_norm: 45.86208865702616
grad_norm: 0.0016569011200549847 tk: 1 x_norm: 54.07100792786304
grad_norm: 0.00044656539992660735 tk: 1 x_norm: 60.05818912308936
grad_norm: 9.270631665087767e-05 tk: 1 x_norm: 63.51391179052652
grad_norm: 2.488990999355918e-05 tk: 1 x_norm: 65.32081185083257
grad_norm: 6.906996095546317e-06 tk: 1 x_norm: 66.52340732805047
grad_norm: 1.2182757880221725e-06 tk: 1 x_norm: 67.1469925290211
grad_norm: 9.244605649907366e-08 tk: 1 x_norm: 67.27916202509665
最小值: 0.058278	耗时: 2.066143s


### CVXPY-计算 optimal value

In [49]:
# D=500
# x = cp.Variable(n)
# # objective = cp.Minimize(sum(cp.log1p(cp.exp(-b*(A@x))))/m + 1/(100*m)* x.T@x)
# # objective = cp.Minimize(sum(cp.log1p(cp.exp(-b*(A@x))))/m)
# objective = cp.Minimize(cp.norm(x)**2 + sum(cp.log1p(cp.exp(-b*(A@x))))/m)
# constraints = [cp.norm(x)<=D/2]
# problem = cp.Problem(objective,constraints)
# result = problem.solve()
# print(f'最优值: {f(x.value)}')

最优值: 0.6931471805599453


/home/lzq22/anaconda3/envs/optim/lib/python3.8/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 8 times so far.

  warnings.warn(msg, UserWarning)
